In [71]:
# %load data.py
# %load data.py
""" Les données sont mieux nettoyé avec ces codes, mais il reste encore des petite faute
"""
from __future__ import print_function

import os
import random
import re

import numpy as np
import csv
import config_tf as config


##############




stop_word = ['?', '.']
def clearing_word(word):
    word = re.sub('\x8e', 'é', word)
    word = re.sub('\x88', 'à', word)
    word = re.sub('\x9d', 'ù', word)
    word = re.sub('\x8f', 'è', word)
    word = re.sub('\x9e', 'û', word)
    word = re.sub('\x90', 'ê', word)
    word = re.sub('\x99', 'ô', word)
    word = re.sub('\x94', 'î', word)
   # word = re.sub('\x8f', 'è', word)
    word = re.sub('\x8d', 'ç', word)
    word = re.sub('õ', '', word)
    word = re.sub('Ê', '', word)
    word = re.sub('[?,.,!, \,,  %]', '', word)
    if word == 'û' or word == 'v' or word == 'é':
        word = ''
    if word =="2017êles":
        word = "2017"
    if "ênox" in word:
        word ="nox"
    return word
    
def clearing(pharse):
    """
    Arg: Data is a list of questions or answers
    Return: clean questions et answers 
    """
    #clean_data = []
    pharses =[]
    for word in pharse.strip().lower().split(' '):
       # line= data[i].lower().split(' ')
        
        #for word in line:
        word = clearing_word(word)
        if word not in stop_word and len(word) !=0:       
            pharses.append(word)
        
    return ' '.join(pharses)



def get_all_convos():
    convos = []
    file1 = 'chatbot_tout_corpus.txt'
    file2 = 'convos_120_nettoye.txt'
    liste_file = [file1, file2]
    for file in liste_file:
        with open(file) as f:
            i=0
            for line in f:
                if i%2==0:
                    question = clearing(line)
                    if '++++' in question:
                         question = question[9:]
                else:
                    answer = clearing(line)
                    if '++++' in answer:
                        answer = answer[9:]
                    convos.append([question, answer])
                i+=1
        f.close()
    return convos

def train_test():
    convos = get_all_convos()
    trains, tests = [], []
    rang = len(convos)
    index_of_train = random.sample(range(rang), int(0.8*rang))
    for i in range(rang):
        if i in index_of_train:
            trains.append(convos[i])
        else:
            tests.append(convos[i])
    save_train_test_in_file(trains, False)
    save_train_test_in_file(tests, True)
    
def save_train_test_in_file(CONVOS, answer=True):
    if not answer:
        out_path1 = os.path.join(config.PROCESSED_PATH, 'train_question.txt')
        out_path2 = os.path.join(config.PROCESSED_PATH, 'train_answer.txt')
    else: 
        out_path1 = os.path.join(config.PROCESSED_PATH, 'test_question.txt')
        out_path2 = os.path.join(config.PROCESSED_PATH, 'test_answer.txt')
    question, answer = [], []
    for i in range(len(CONVOS)):
        question.append(CONVOS[i][0])
        answer.append(CONVOS[i][1])
    with open(out_path1, 'w') as f1:
        for i in range(len(question)):
            f1.write(question[i]+'\n')
    f1.close()
    with open(out_path2, 'w') as f2:
        for i in range(len(answer)):
            f2.write(answer[i]+'\n')
    f2.close()
        
            








def question_answers(convos):
    """ Divide the dataset into two sets: questions and answers. """
    questions, answers = [], []
    for convo in convos:
        question = [];answer = []
        for word in convo[0].split():
            if word not in config.STOPWORDS and word not in "([.,!?\"'-<>:;)(])":
                question.append(word)
        for word in convo[-1].split():
            if word not in config.STOPWORDS and word not in "([.,!?\"'-<>:;)(])":
                answer.append(word)
        question = ' '.join(word for word in question)
        answer = ' '.join(word for word in answer)
        questions.append(question)
        answers.append(answer)
    assert len(questions) == len(answers)
    return questions, answers


        
def prepare_dataset(questions, answers):
    make_dir(config.PROCESSED_PATH)
    path1= os.path.join(config.PROCESSED_PATH, 'question_train.txt')
    path2 = os.path.join(config.PROCESSED_PATH, 'answer_train.txt')
    try: 
        os.remove(path1);   os.remove(path2)
    except FileNotFoundError:
        pass
    with open(path1, 'w') as f:
        for i in range(len(questions)):
            f.write(questions[i]+'\n')
    f.close()
    with open(path2, 'w') as f:
        for i in range(len(answers)):
            f.write(answers[i]+'\n')
    f.close()

def make_dir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.mkdir(path)
    except OSError:
        pass





def build_vocab():
    convos = get_all_convos()
    enc_vocab = ['<pad>', '<unk>', '<s>', '<\s>']
    dec_vocab = ['<pad>', '<unk>', '<s>', '<\s>']
    enc, dec = 4, 4
    for i in range(len(convos)):
        for word in convos[i][0].strip().split():
            if word not in enc_vocab:
                enc_vocab.append(word)
                enc+=1
        for word in convos[i][1].strip().split():
            if word not in dec_vocab:
                dec_vocab.append(word)
                dec+=1
    with open('config_tf.py', 'a') as cf:
              cf.write('DEC_VOCAB = '+str(dec)+'\n')
              cf.write('ENC_VOCAB = '+str(enc)+'\n')
    cf.close()
    inv_enc_vocab = {enc_vocab[i]: i for i in range(len(enc_vocab))}
    inv_dec_vocab = {dec_vocab[i]: i for i in range(len(dec_vocab))}
    return enc_vocab, dec_vocab, inv_enc_vocab, inv_dec_vocab           

def sentence2id(vocab, line):
    return [vocab[word] for word in line.split() if word in vocab]

def token2id(data, inv_enc_vocab, inv_dec_vocab):
    """ 
    arg: data may be trains list or test list
    return ids. """
    
    questions2id =[]
    answers2id = []
    for i in range(len(data)):
        questions2id.append(sentence2id(inv_enc_vocab, data[i][0]))
        answers2id.append(sentence2id(inv_dec_vocab, data[i][1]))
    return questions2id, answers2id

def load_data(enc_filename, dec_filename, inv_enc_vocab, inv_dec_vocab):
    """
    load train or test data"""

    encode_file = open(os.path.join(config.PROCESSED_PATH, enc_filename), 'r')
    decode_file = open(os.path.join(config.PROCESSED_PATH, dec_filename), 'r')
    encode, decode = encode_file.readline(), decode_file.readline()
    data_buckets = []
    convos =[]
    while encode and decode:
        convos.append([encode, decode])
        encode, decode = encode_file.readline(), decode_file.readline()
    questions2id, answers2id = token2id(convos, inv_enc_vocab, inv_dec_vocab)
    for i in range(len(questions2id)):
        encode_ids =questions2id[i]
        decode_ids = answers2id[i]
        if len(encode_ids) <= config.BUCKETS[-1][0] and len(decode_ids) <= config.BUCKETS[-1][-1]:
                data_buckets.append([encode_ids, decode_ids])
    print("load {} sentences from data".format(i))
    return data_buckets

def _pad_input(input_, size):
    return input_ + [config.PAD_ID] * (size - len(input_))

def _reshape_batch(inputs, size, batch_size):
    """ Create batch-major inputs. Batch inputs are just re-indexed inputs
    """
    batch_inputs = []
    for length_id in range(size):
        batch_inputs.append(np.array([inputs[batch_id][length_id]
                                    for batch_id in range(batch_size)], dtype=np.int32))
    return batch_inputs

def dictionnary_for_chat():
    out_file = open(os.path.join(config.PROCESSED_PATH, 'dictionnary.txt'), 'w')
    with open('thes_fr.txt') as f:
        for line in f:
            line = re.split("[()]",line)[-1]
            line = line.strip('|').strip('1').strip().split('|')
            line = [i for i in line if i != '1']
            out_file.write('|'.join(str(id_) for id_ in line) + '\n')
    out_file.close() 
    
def construct_dict(lang):
    """
    return list des vocabulaire"""
    in_file = open(os.path.join(config.PROCESSED_PATH, 'dictionnary.txt'), 'r')
    lines = []
    for word in lang.word2index:
        for line in in_file:
            line = line.strip().split('|')
            if word in line:
                line.insert(word,0)
                lines.append(line)
                break
    return lines
    

def get_batch(data_buckets, bucket_id, batch_size=1):
    """ Return one batch to feed into the model """
    # only pad to the max length of the bucket
    encoder_size, decoder_size = config.BUCKETS[bucket_id]
    encoder_inputs, decoder_inputs = [], []

    for _ in range(batch_size):
        encoder_input, decoder_input = random.choice(data_buckets)
        # pad both encoder and decoder, reverse the encoder
        # So, encoder start by 0, 0, ...
        encoder_inputs.append(list(reversed(_pad_input(encoder_input, encoder_size))))
        decoder_inputs.append(_pad_input(decoder_input, decoder_size))

    # now we create batch-major vectors from the data selected above.
    batch_encoder_inputs = _reshape_batch(encoder_inputs, encoder_size, batch_size)
    batch_decoder_inputs = _reshape_batch(decoder_inputs, decoder_size, batch_size)

    # create decoder_masks to be 0 for decoders that are padding.
    batch_masks = []
    for length_id in range(decoder_size):
        batch_mask = np.ones(batch_size, dtype=np.float32)
        for batch_id in range(batch_size):
            # we set mask to 0 if the corresponding target is a PAD symbol.
            # the corresponding decoder is decoder_input shifted by 1 forward.
            if length_id < decoder_size - 1:
                target = decoder_inputs[batch_id][length_id + 1]
            if length_id == decoder_size - 1 or target == config.PAD_ID:
                batch_mask[batch_id] = 0.0
        batch_masks.append(batch_mask)
    return batch_encoder_inputs, batch_decoder_inputs, batch_masks

if __name__ == '__main__':
    train_test()

In [72]:
enc_vocab, dec_vocab, inv_enc_vocab, inv_dec_vocab = build_vocab()
train = load_data('train_question.txt', 'train_answer.txt', inv_enc_vocab, inv_dec_vocab)
test = load_data('test_question.txt', 'test_answer.txt', inv_enc_vocab, inv_dec_vocab)

load 567 sentences from data
load 141 sentences from data


In [102]:
idx = random.sample(range(len(train)), 1)
data1 = train[idx[0]]
question =  [enc_vocab[ix] for ix in data1[0]]
answer = [dec_vocab[ix] for ix in data1[1]]
print(question, '   ', answer)

['quel', 'coût']     ['cela', 'dépend', 'de', 'vos', 'besoin', 'contactez-nous', 'au', '#num_telephone#']


In [17]:
idx = random.sample(range(len(test)), 1)
data1 = test[idx[0]]
question =  [enc_vocab[ix] for ix in data1[0]]
answer = [dec_vocab[ix] for ix in data1[1]]
print(question, '   ', answer)

'<pad>'

In [40]:
convos = get_all_convos()
for i in range(20):
    print(random.choice(convos))

[" quel est l'historique des trajets", 'voici #histo_trip# des trajets']
[' bonsoir', 'bonsoir']
[' carnet de santé du véhicule', "le carnet de santé du véhicule permet de considérer tous les événements survenus depuis l'entrée du véhicule dans votre parc c'est un document précieux qui apporte transparence et confiance nous avons développé les technologies utiles et un module spécifiques pour cela"]
[" je veux connaître la date d'achat d'un véhicule", 'pas de souci quel est le véhicule concerné']
[' il y a des problèmes', '#oui/non#']
[' respectons-nous les échéances', 'il convient de préciser la nature des échéances qui vous importent révision maintenance restitution']
[' combien de véhicule composent ma flotte', '#nb_vehicule#']
[' quelle est la puissance du véhicule', 'la puissance fiscale du #vin# est #cv# la puissance fiscale du #vin# est #cv#']
[' nous avons des', 'oui posez votre question']
[' je veux le détail de la car policy', 'votre car policy est disponible dans le coffre-f

In [68]:
a, b =token2id(convos[1], inv_enc_vocab, inv_dec_vocab)

In [64]:
sentence2id( inv_enc_vocab, convos[1][0])

[9, 5, 6, 7, 8]

In [70]:
convos[1]

[" avons-nous des véhicules à l'étranger", 'je vérifie cela tout de suite']

In [66]:
a1= a[1]
b1=b[1]
question =  [enc_vocab[ix] for ix in a1]
answer = [dec_vocab[ix] for ix in b1]
print(question, '   ', answer)

[]     []


In [69]:
a

[[], []]

In [44]:
token2id(t, inv_enc_vocab, inv_dec_vocab)

([[], []], [[70], []])

In [47]:
inv_enc_vocab

{'<pad>': 0,
 '<unk>': 1,
 '<s>': 2,
 '<\\s>': 3,
 '+': 4,
 'des': 5,
 'véhicules': 6,
 'à': 7,
 "l'étranger": 8,
 'avons-nous': 9,
 'combien': 10,
 'de': 11,
 'nous': 12,
 'avons': 13,
 'hors': 14,
 'périmètre': 15,
 'délai': 16,
 'livraison': 17,
 'du': 18,
 'véhicule': 19,
 'boîtiers': 20,
 'mode': 21,
 'paiement': 22,
 'quel': 23,
 'solution': 24,
 'quelles': 25,
 'sont': 26,
 'les': 27,
 'moyens': 28,
 'adresse': 29,
 'facturation': 30,
 'où': 31,
 'est': 32,
 'la': 33,
 'facture': 34,
 'ma': 35,
 'je': 36,
 'veux': 37,
 'oui': 38,
 'imprimer': 39,
 'boitier': 40,
 'service': 41,
 'le': 42,
 'hs': 43,
 'si': 44,
 'ne': 45,
 'fonctionne': 46,
 'plus': 47,
 'que': 48,
 'dois-je': 49,
 'faire': 50,
 'boitiers': 51,
 'pas': 52,
 'reconnus': 53,
 'temps': 54,
 'pour': 55,
 'installer': 56,
 'boîtier': 57,
 'un': 58,
 'se': 59,
 'connecte': 60,
 'suis': 61,
 'bloqué': 62,
 'apparaît': 63,
 'grisé': 64,
 'semble': 65,
 'déconnecté': 66,
 'communique': 67,
 'paramêtrer': 68,
 'paramêtrage

In [50]:
'avons-nous' in inv_enc_vocab

True